In [1]:
import json
import pandas as pd
import numpy as np
from ipywidgets import widgets
from numpy.linalg import norm
from neo4j import GraphDatabase

In [2]:
with open('config.json', 'r') as f:
    credentials = json.load(f)

uri = credentials['uri']
user = credentials['user']
password = credentials['password']
driver = GraphDatabase.driver(uri, auth=(user, password))
#session = driver.session()

In [3]:
def get_options(driver):
    session = driver.session()
    result = []
    query = """
            MATCH (u:User)-[r]-(n:Symptom)
            RETURN DISTINCT n.symptom AS symptom, COUNT(u) AS ocurrency
            ORDER BY ocurrency DESC
        """
    with session as s:
        tx = s.begin_transaction()
        for row in tx.run(query):
            try:
                if row:
                    symptom = (row[0]).strip()
                    ocurrency = row[1]
                    node_dict = {"symptom": symptom, "ocurrency": ocurrency}
                    result.append(node_dict)
            except Exception as e:
                print(e, row)
        tx.commit()
    return result

In [4]:
options = get_options(driver)
options_list = [x['symptom'] for x in options]
options_list_top_10 = []
for i in range(0, 20): 
    options_list_top_10.append(options_list[i])

In [5]:
def diagnose(driver,symptoms):
    possible_diseases = []
    session = driver.session()
    query = """
        WITH {0} AS new_symptoms
        MATCH (p:Patient)-[s:SIMILAR]-(o:Patient)
        WHERE ANY(symptom IN p.symptoms WHERE symptom IN new_symptoms)
        RETURN DISTINCT o.disease AS disease, s.score AS similarity
        ORDER BY similarity DESC
        LIMIT 20
    """
    query = query.format(symptoms)
    with driver.session() as session:
        tx = session.begin_transaction()
        for row in tx.run(query):
            try:
                if row:
                    disease = (row[0]).strip()
                    similarity = row[1]
                    node_dict = {"disease": disease, "similarity": similarity}
                    possible_diseases.append(node_dict)
            except Exception as e:
                print(e, row)
        tx.commit()
    return possible_diseases
    

In [6]:
widget_seleccion = widgets.SelectMultiple(
    options=options_list_top_10,
    value=[options_list_top_10[0]],
    description='Selecciona opciones:'
)

widget_seleccion.layout.height = '300px'
display(widget_seleccion)


SelectMultiple(description='Selecciona opciones:', index=(0,), layout=Layout(height='300px'), options=('fatigu…

In [9]:
symptoms = [x for x in widget_seleccion.value]
print("Evaluating your symptoms: ", symptoms)
result = diagnose(driver=driver, symptoms=symptoms)
result

Evaluating your symptoms:  ['skin rash', 'joint pain']


[{'disease': 'gastroenteritis', 'similarity': 1.0},
 {'disease': 'dengue', 'similarity': 1.0},
 {'disease': 'hepatitis e', 'similarity': 1.0},
 {'disease': 'hepatitis d', 'similarity': 1.0},
 {'disease': 'drug reaction', 'similarity': 1.0},
 {'disease': 'fungal infection', 'similarity': 1.0},
 {'disease': 'hepatitis a', 'similarity': 1.0},
 {'disease': 'chicken pox', 'similarity': 1.0},
 {'disease': 'osteoarthristis', 'similarity': 1.0},
 {'disease': 'aids', 'similarity': 1.0},
 {'disease': 'acne', 'similarity': 1.0},
 {'disease': 'psoriasis', 'similarity': 1.0},
 {'disease': 'impetigo', 'similarity': 1.0},
 {'disease': 'dengue', 'similarity': 0.9285714285714286},
 {'disease': 'hepatitis e', 'similarity': 0.9230769230769231},
 {'disease': 'chicken pox', 'similarity': 0.9090909090909091},
 {'disease': 'hepatitis a', 'similarity': 0.9090909090909091},
 {'disease': 'hepatitis d', 'similarity': 0.8888888888888888},
 {'disease': 'psoriasis', 'similarity': 0.8333333333333334},
 {'disease': '